In [1]:
!pip -q install transformers
!pip -q install git+https://github.com/mansaluke/newsai.git

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 3.8MB 13.8MB/s 
     |████████████████████████████████| 890kB 33.0MB/s 
     |████████████████████████████████| 1.1MB 24.6MB/s 
     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 256kB 14.6MB/s 
     |████████████████████████████████| 153kB 13.0MB/s 


In [0]:
%matplotlib inline

In [4]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from newsai import dfconvert
from newsai.utils.nlp import *
import transformers
from tqdm import tqdm
from IPython.display import display, HTML
from transformers import pipeline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import torch
torch.cuda.is_available()

True

In [6]:
from newsai import _DATA_PATH
if not os.path.exists(_DATA_PATH):
  import requests
  URL = 'https://raw.githubusercontent.com/mansaluke/newsai/master/data/sample_historicals.csv'
  _DATA_PATH = dfconvert.mkdir_p(os.path.expanduser("~/.newsai"))
  response = requests.get(URL, stream=True)
  response.raise_for_status()
  file_size = int(response.headers["Content-Length"])

  with tqdm(total=file_size) as pbar:
    with open(os.path.join(_DATA_PATH, os.path.basename(URL)), 'wb') as handle:
        for block in response.iter_content(1024):
            handle.write(block)
            pbar.update(1024)

2728960it [00:00, 10573469.94it/s]                            


In [86]:
fpath = os.path.join(_DATA_PATH, 'sample_historicals.csv')
df = pd.read_csv(fpath, encoding = "UTF-8", parse_dates=['date'])
df = remove_null_rows(df, ["H1", "H2"])

2020-05-17 13:48:25,577 - newsai.utils.nlp - INFO - Removing 1010 rows with nulls


In [134]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("bart-large-cnn")

In [0]:
def dataframe_text_summarizer(txt, var=0.1):
    txt = str(txt)
    txt = tokenizer.encode("summarize: " + txt, return_tensors="pt", max_length=512)
    min_length = max(round(len(txt[0]) * (var)), 1)
    max_length = max(round(len(txt[0]) * (1-var)), 1)
    return tokenizer.decode(
        model.generate(txt, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True).tolist()[0]
    )

In [0]:
df = df[:5]

In [137]:
p = %prun -r df['H1_summary'] = df.H1.apply(lambda x: dataframe_text_summarizer(x))

In [138]:
p.sort_stats('time', 'cumtime').print_stats()

         590026 function calls (568440 primitive calls) in 26.132 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12023   20.241    0.002   20.241    0.002 {method 'matmul' of 'torch._C._TensorBase' objects}
        5    1.934    0.387    1.934    0.387 {method 'uniform_' of 'torch._C._TensorBase' objects}
     5705    0.619    0.000    0.619    0.000 {method 'index_select' of 'torch._C._TensorBase' objects}
     5832    0.369    0.000    0.369    0.000 {built-in method bmm}
    12023    0.337    0.000   20.757    0.002 functional.py:1591(linear)
     4278    0.284    0.000    0.284    0.000 {built-in method cat}
     2916    0.244    0.000    8.862    0.003 modeling_bart.py:577(forward)
      119    0.170    0.001    0.170    0.001 {built-in method topk}
21446/124    0.149    0.000   23.031    0.186 module.py:540(__call__)
     4528    0.140    0.000    0.140    0.000 {built-in method layer_norm}
     

In [130]:
nlp_sentence_classif = pipeline('sentiment-analysis')

In [0]:
df['H1_sentiment'] = df.H1.apply(lambda x: nlp_sentence_classif(x))

In [133]:
display(HTML(df[['H1', 'H1_summary', 'H1_sentiment']].head(10).to_html()))

,H1,H1_summary,H1_sentiment
0,"Attacks have been traditionally underreported, but fighting the scourge in hate crimes begins with better data, experts say.","attacks have traditionally been underreported, but fighting hate crimes begins with better data. fighting hate crimes begins with better data","[{'label': 'NEGATIVE', 'score': 0.992476224899292}]"
1,"He deftly manipulated electronic images to produce otherworldly, sometimes jarring visions. He also founded, with his wife, a landmark performance space.","he deftly manipulated electronic images to produce otherworldly, sometimes jarring visions. he also founded, with his wife,","[{'label': 'POSITIVE', 'score': 0.9991649389266968}]"
2,"The past definitely wasn’t past for directors like Quentin Tarantino, Martin Scorsese and Taika Waititi. Each found a way to rewrite history, though some versions were less palatable than others.","the past definitely wasn’t past for directors like Quentin Tarantino, Martin Scorsese and Taika Waititi. each found a way to rewrite history, though some versions were less","[{'label': 'NEGATIVE', 'score': 0.9796326756477356}]"
3,Forget fad diets and fitness gimmicks. Just stick to the basics.,Forget fad diets and fitness gimmicks. stick to the basics,"[{'label': 'NEGATIVE', 'score': 0.9918994903564453}]"
4,"Linda Tracy and Peter Tracy had been married 37 years, before divorcing in 2015. When their sons began planning their own weddings, Mom and Dad rediscovered love.","Linda Tracy and Peter Tracy had been married 37 years, before divorcing in 2015. When their sons began planning their own weddings, Mom and Dad re","[{'label': 'POSITIVE', 'score': 0.9993278980255127}]"


In [0]:
# use h1 to train h2?